# Electricity

In [2]:
import pandas as pd
"""
This script reads a CSV file into a pandas DataFrame, preprocesses the data using a custom preprocessing function,
and combines the training and validation sets. It then prints the shapes of the resulting datasets.

Functions:
    preprocess_data(df): Preprocesses the input DataFrame and returns normalized training, validation, and test sets.

Variables:
    df (DataFrame): The input data read from a CSV file.
    output_names (list): A list containing the name of the output variable.
    X_train_normalized (ndarray): Normalized training features.
    y_train_normalized (ndarray): Normalized training labels.
    X_val_normalized (ndarray): Normalized validation features.
    y_val_normalized (ndarray): Normalized validation labels.
    X_test_normalized (ndarray): Normalized test features.
    y_test_normalized (ndarray): Normalized test labels.
    X_train_val_normalized (ndarray): Combined normalized training and validation features.
    y_train_val_normalized (ndarray): Combined normalized training and validation labels.
"""
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')

output_names = ['Electricity Consumption']

from FDCS_1_Elec_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 35)
Shape of y_train_normalized: (1272, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (1608, 35)
Shape of y_train_val_normalized: (1608, 1)


## Models

In [2]:
from FDCS_1_models_FS import run_model
"""
This script runs a model using the `run_model` function from the `FDCS_1_models_FS` module and stores the predictions and true values for the test set.

Functions:
    run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names): 
        Runs the model on the provided training and test data.

Variables:
    pred_true_TestSet (tuple): 
        A tuple containing the predictions and true values for the test set.
"""
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Execution time: 268.85 seconds


,Model,True_Electricity Consumption,Pred_Electricity Consumption
0,KNeighborsRegressor (Without),0.018988,0.008834
1,KNeighborsRegressor (Without),0.019199,0.009238
2,KNeighborsRegressor (Without),0.023504,0.009220
3,KNeighborsRegressor (Without),0.018734,0.008816
4,KNeighborsRegressor (Without),0.016716,0.007461
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.350507,0.290554
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.317073,0.303773
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.372739,0.303242
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.227587,0.207971


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
29,RandomForestRegressor (Wrapper - Forward Selec...,Electricity Consumption,0.0432,0.0652
1,RandomForestRegressor (Without),Electricity Consumption,0.0436,0.0659
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Electricity Consumption,0.0440,0.0673
9,RandomForestRegressor (Filter - Mutual Informa...,Electricity Consumption,0.0447,0.0672
3,XGBRegressor (Without),Electricity Consumption,0.0450,0.0686
27,XGBRegressor (Wrapper - RFE (LinearRegression)),Electricity Consumption,0.0452,0.0699
11,XGBRegressor (Filter - Mutual Information),Electricity Consumption,0.0453,0.0677
31,XGBRegressor (Wrapper - Forward Selection),Electricity Consumption,0.0454,0.0700
33,RandomForestRegressor (Hybrid - Filter (Select...,Electricity Consumption,0.0454,0.0678
23,XGBRegressor (Embedded - ElasticNet),Electricity Consumption,0.0458,0.0677


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_elec.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Electricity Consumption']])
pred_true_TestSet['Pred_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Electricity Consumption']])
pred_true_TestSet.to_csv('FDCS_1_Elec_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Electricity Consumption,Pred_Electricity Consumption
0,KNeighborsRegressor (Without),172.61,136.996680
1,KNeighborsRegressor (Without),173.35,138.411519
2,KNeighborsRegressor (Without),188.45,138.350770
3,KNeighborsRegressor (Without),171.72,136.933611
4,KNeighborsRegressor (Without),164.64,132.180434
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,1335.41,1125.124681
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,1218.14,1171.490501
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,1413.39,1169.629112
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,904.27,835.466226


## LSTM

In [2]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_1_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-06 19:14:44.564813: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 19:14:44.567000: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-06 19:14:44.607889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-06 19:14:44.608610: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-06 19:14:45.318545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100


2024-03-06 19:14:47.149998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 19:14:47.151420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 19:14:47.152957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - ETA: 0s - loss: 0.0965

2024-03-06 19:14:50.765420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 19:14:50.767269: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 19:14:50.768624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-03-06 19:55:55.212684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 19:55:55.214464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 19:55:55.215683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - 4s 122ms/step - loss: 0.0975 - val_loss: 0.0209
Epoch 2/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0416 - val_loss: 0.0157
Epoch 3/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0315 - val_loss: 0.0281
Epoch 4/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0268 - val_loss: 0.0161
Epoch 5/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0239 - val_loss: 0.0159
Epoch 6/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0230 - val_loss: 0.0193
Epoch 7/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0221 - val_loss: 0.0135
Epoch 8/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0218 - val_loss: 0.0132
Epoch 9/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0204 - val_loss: 0.0124
Epoch 10/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0194 - val_loss: 0.0107
Epoch 11/100
9/9 [============

2024-03-06 20:01:22.490545: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-06 20:01:22.492230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-06 20:01:22.493486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 8ms/step
Execution time: 2796.85 seconds


,Model,True,Pred
0,LSTM + Without,0.020704,-0.005654
1,LSTM + Without,0.020183,0.000094
2,LSTM + Without,0.018472,0.001155
3,LSTM + Without,0.023481,-0.005301
4,LSTM + Without,0.020237,-0.002683
...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.350507,0.263351
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.317073,0.317926
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.372739,0.291809
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.227587,0.204010


In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Electricity Consumption,0.0481,0.0729
4,LSTM + Embedded - Tree Importance (Extra Trees),Electricity Consumption,0.0496,0.0737
3,LSTM + Embedded - Lasso,Electricity Consumption,0.0518,0.0799
2,LSTM + Filter - Mutual Information,Electricity Consumption,0.0564,0.0816
1,LSTM + Filter - Correlation,Electricity Consumption,0.0584,0.0877
5,LSTM + Embedded - ElasticNet,Electricity Consumption,0.0588,0.0832
6,LSTM + Wrapper - RFE (LinearRegression),Electricity Consumption,0.0609,0.0837
0,LSTM + Without,Electricity Consumption,0.0701,0.1012
7,LSTM + Wrapper - Forward Selection,Electricity Consumption,0.0720,0.1048


In [6]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_elec.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_1_Elec_pred_true_TestSet_LSTM.csv')
pred_true_TestSet


,Model,True,Pred,True_Electricity Consumption,Pred_Electricity Consumption
0,LSTM + Without,0.020704,-0.005654,178.63,86.177032
1,LSTM + Without,0.020183,0.000094,176.80,106.338310
2,LSTM + Without,0.018472,0.001155,170.80,110.061996
3,LSTM + Without,0.023481,-0.005301,188.37,87.416748
4,LSTM + Without,0.020237,-0.002683,176.99,96.600403
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.350507,0.263351,1335.41,1029.710327
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.317073,0.317926,1218.14,1221.133057
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.372739,0.291809,1413.39,1129.528564
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.227587,0.204010,904.27,821.573486


# Temp

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')

output_names = ['Temperature Closer to Evaporator (C)']

from FDCS_1_Temp_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 35)
Shape of y_train_normalized: (1272, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (1608, 35)
Shape of y_train_val_normalized: (1608, 1)


## Models

In [2]:
from FDCS_1_models_FS import run_model
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Execution time: 227.80 seconds


,Model,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,KNeighborsRegressor (Without),0.303235,0.483033
1,KNeighborsRegressor (Without),0.460916,0.479723
2,KNeighborsRegressor (Without),0.516173,0.540790
3,KNeighborsRegressor (Without),0.528302,0.557271
4,KNeighborsRegressor (Without),0.516173,0.605578
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.314016,0.304564
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.212938,0.172490
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.168464,0.164694
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.168464,0.177715


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
31,XGBRegressor (Wrapper - Forward Selection),Temperature Closer to Evaporator (C),0.0468,0.0625
27,XGBRegressor (Wrapper - RFE (LinearRegression)),Temperature Closer to Evaporator (C),0.0482,0.0641
1,RandomForestRegressor (Without),Temperature Closer to Evaporator (C),0.0489,0.0641
3,XGBRegressor (Without),Temperature Closer to Evaporator (C),0.0493,0.0655
29,RandomForestRegressor (Wrapper - Forward Selec...,Temperature Closer to Evaporator (C),0.0495,0.0646
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Temperature Closer to Evaporator (C),0.0523,0.0665
34,MLPRegressor (Hybrid - Filter (SelectKBest) + ...,Temperature Closer to Evaporator (C),0.0594,0.0749
9,RandomForestRegressor (Filter - Mutual Informa...,Temperature Closer to Evaporator (C),0.0605,0.0780
11,XGBRegressor (Filter - Mutual Information),Temperature Closer to Evaporator (C),0.0610,0.0791
5,RandomForestRegressor (Filter - Correlation),Temperature Closer to Evaporator (C),0.0613,0.0790


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_temp.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Temperature Closer to Evaporator (C)']])
pred_true_TestSet['Pred_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Temperature Closer to Evaporator (C)']])
pred_true_TestSet.to_csv('FDCS_1_Temp_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,KNeighborsRegressor (Without),11.75,13.084102
1,KNeighborsRegressor (Without),12.92,13.059545
2,KNeighborsRegressor (Without),13.33,13.512665
3,KNeighborsRegressor (Without),13.42,13.634952
4,KNeighborsRegressor (Without),13.33,13.993389
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,11.83,11.759864
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,11.08,10.779876
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,10.75,10.722028
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,10.75,10.818647


## LSTM

In [2]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_1_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-07 11:20:46.417590: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 11:20:46.450465: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 11:20:46.628059: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 11:20:46.629572: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 11:20:47.602118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100


2024-03-07 11:20:50.209776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 11:20:50.211919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 11:20:50.213379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - ETA: 0s - loss: 0.1736

2024-03-07 11:20:53.876114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 11:20:53.877926: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 11:20:53.879178: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-03-07 12:04:04.928651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 12:04:04.930359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 12:04:04.931669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - 5s 133ms/step - loss: 0.1024 - val_loss: 0.0515
Epoch 2/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0416 - val_loss: 0.0294
Epoch 3/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0258 - val_loss: 0.0263
Epoch 4/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0182 - val_loss: 0.0174
Epoch 5/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0148 - val_loss: 0.0194
Epoch 6/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0144 - val_loss: 0.0170
Epoch 7/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0140 - val_loss: 0.0177
Epoch 8/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0141 - val_loss: 0.0195
Epoch 9/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0132 - val_loss: 0.0176
Epoch 10/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0148 - val_loss: 0.0155
Epoch 11/100
9/9 [============

2024-03-07 12:09:42.456943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 12:09:42.458552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 12:09:42.459735: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 8ms/step
Execution time: 2934.00 seconds


,Model,True,Pred
0,LSTM + Without,0.292453,0.400391
1,LSTM + Without,0.471698,0.532137
2,LSTM + Without,0.539084,0.541340
3,LSTM + Without,0.539084,0.523193
4,LSTM + Without,0.539084,0.521717
...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.314016,0.274594
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.212938,0.166743
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.168464,0.155955
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.168464,0.176678


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
7,LSTM + Wrapper - Forward Selection,Temperature Closer to Evaporator (C),0.0687,0.0890
3,LSTM + Embedded - Lasso,Temperature Closer to Evaporator (C),0.0756,0.0934
0,LSTM + Without,Temperature Closer to Evaporator (C),0.0759,0.0965
1,LSTM + Filter - Correlation,Temperature Closer to Evaporator (C),0.0777,0.0993
2,LSTM + Filter - Mutual Information,Temperature Closer to Evaporator (C),0.0797,0.0993
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Temperature Closer to Evaporator (C),0.0800,0.0938
6,LSTM + Wrapper - RFE (LinearRegression),Temperature Closer to Evaporator (C),0.0848,0.1112
4,LSTM + Embedded - Tree Importance (Extra Trees),Temperature Closer to Evaporator (C),0.0874,0.1083
5,LSTM + Embedded - ElasticNet,Temperature Closer to Evaporator (C),0.1002,0.1191


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_temp.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_1_Temp_pred_true_TestSet_LSTM.csv')
pred_true_TestSet


,Model,True,Pred,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,LSTM + Without,0.292453,0.400391,11.67,12.470900
1,LSTM + Without,0.471698,0.532137,13.00,13.448458
2,LSTM + Without,0.539084,0.541340,13.50,13.516743
3,LSTM + Without,0.539084,0.523193,13.50,13.382095
4,LSTM + Without,0.539084,0.521717,13.50,13.371140
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.314016,0.274594,11.83,11.537488
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.212938,0.166743,11.08,10.737235
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.168464,0.155955,10.75,10.657187
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.168464,0.176678,10.75,10.810950


# Humd

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')

output_names = ['Humidity Closer to Evaporator (%)']

from FDCS_1_Humd_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 35)
Shape of y_train_normalized: (1272, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (1608, 35)
Shape of y_train_val_normalized: (1608, 1)


## Models

In [2]:
from FDCS_1_models_FS import run_model
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Execution time: 305.31 seconds


,Model,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,KNeighborsRegressor (Without),0.443696,0.545441
1,KNeighborsRegressor (Without),0.331408,0.511214
2,KNeighborsRegressor (Without),0.299326,0.443095
3,KNeighborsRegressor (Without),0.270132,0.428123
4,KNeighborsRegressor (Without),0.248637,0.433001
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.462624,0.332914
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.307347,0.321746
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.505294,0.323413
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.582932,0.330150


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
35,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,Humidity Closer to Evaporator (%),0.0634,0.0858
33,RandomForestRegressor (Hybrid - Filter (Select...,Humidity Closer to Evaporator (%),0.0658,0.0876
9,RandomForestRegressor (Filter - Mutual Informa...,Humidity Closer to Evaporator (%),0.0673,0.0885
27,XGBRegressor (Wrapper - RFE (LinearRegression)),Humidity Closer to Evaporator (%),0.0677,0.0887
11,XGBRegressor (Filter - Mutual Information),Humidity Closer to Evaporator (%),0.0679,0.0896
7,XGBRegressor (Filter - Correlation),Humidity Closer to Evaporator (%),0.0687,0.0926
5,RandomForestRegressor (Filter - Correlation),Humidity Closer to Evaporator (%),0.0687,0.0916
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Humidity Closer to Evaporator (%),0.0692,0.0898
1,RandomForestRegressor (Without),Humidity Closer to Evaporator (%),0.0700,0.0911
3,XGBRegressor (Without),Humidity Closer to Evaporator (%),0.0701,0.0923


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_humd.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Humidity Closer to Evaporator (%)']])
pred_true_TestSet['Pred_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Humidity Closer to Evaporator (%)']])
pred_true_TestSet.to_csv('FDCS_1_Humd_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,KNeighborsRegressor (Without),75.08,78.251389
1,KNeighborsRegressor (Without),71.58,77.184534
2,KNeighborsRegressor (Without),70.58,75.061261
3,KNeighborsRegressor (Without),69.67,74.594606
4,KNeighborsRegressor (Without),69.00,74.746656
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,75.67,71.626922
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,70.83,71.278827
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,77.00,71.330797
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,79.42,71.540765


## LSTM

In [3]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_1_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-07 12:17:28.065307: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 12:17:28.067184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 12:17:28.107964: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 12:17:28.108741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 12:17:28.864259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100


2024-03-07 12:17:30.762607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 12:17:30.764781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 12:17:30.766267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - ETA: 0s - loss: 0.0749

2024-03-07 12:17:34.401105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 12:17:34.402980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 12:17:34.404400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-03-07 12:58:18.888905: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 12:58:18.890777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 12:58:18.892272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9/9 [==============================] - 5s 128ms/step - loss: 0.0652 - val_loss: 0.0232
Epoch 2/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0193 - val_loss: 0.0215
Epoch 3/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0174 - val_loss: 0.0223
Epoch 4/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0152 - val_loss: 0.0177
Epoch 5/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0137 - val_loss: 0.0162
Epoch 6/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0126 - val_loss: 0.0158
Epoch 7/100
9/9 [==============================] - 0s 43ms/step - loss: 0.0129 - val_loss: 0.0138
Epoch 8/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0113 - val_loss: 0.0138
Epoch 9/100
9/9 [==============================] - 0s 44ms/step - loss: 0.0111 - val_loss: 0.0121
Epoch 10/100
9/9 [==============================] - 0s 42ms/step - loss: 0.0110 - val_loss: 0.0123
Epoch 11/100
9/9 [============

2024-03-07 13:04:36.020565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 13:04:36.022190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 13:04:36.023354: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 8ms/step
Execution time: 2826.80 seconds


,Model,True,Pred
0,LSTM + Without,0.419634,0.588832
1,LSTM + Without,0.358357,0.422273
2,LSTM + Without,0.326275,0.405014
3,LSTM + Without,0.312801,0.385457
4,LSTM + Without,0.299326,0.392229
...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.462624,0.531777
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.307347,0.529864
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.505294,0.516157
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.582932,0.502976


In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)

,Model_FS,Output,MAE,RMSE
6,LSTM + Wrapper - RFE (LinearRegression),Humidity Closer to Evaporator (%),0.0793,0.1038
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Humidity Closer to Evaporator (%),0.0815,0.1026
5,LSTM + Embedded - ElasticNet,Humidity Closer to Evaporator (%),0.0876,0.1147
3,LSTM + Embedded - Lasso,Humidity Closer to Evaporator (%),0.0885,0.1068
1,LSTM + Filter - Correlation,Humidity Closer to Evaporator (%),0.0945,0.1337
7,LSTM + Wrapper - Forward Selection,Humidity Closer to Evaporator (%),0.0974,0.1203
2,LSTM + Filter - Mutual Information,Humidity Closer to Evaporator (%),0.1017,0.1249
0,LSTM + Without,Humidity Closer to Evaporator (%),0.1018,0.1269
4,LSTM + Embedded - Tree Importance (Extra Trees),Humidity Closer to Evaporator (%),0.1093,0.1353


In [5]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_humd.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_1_Humd_pred_true_TestSet_LSTM.csv')
pred_true_TestSet

,Model,True,Pred,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,LSTM + Without,0.419634,0.588832,74.33,79.603897
1,LSTM + Without,0.358357,0.422273,72.42,74.412254
2,LSTM + Without,0.326275,0.405014,71.42,73.874283
3,LSTM + Without,0.312801,0.385457,71.00,73.264687
4,LSTM + Without,0.299326,0.392229,70.58,73.475777
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.462624,0.531777,75.67,77.825478
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.307347,0.529864,70.83,77.765854
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.505294,0.516157,77.00,77.338623
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.582932,0.502976,79.42,76.927750
